# Chroma DB

In [3]:
#!pip install chromadb

In [4]:
import chromadb

chroma_client = chromadb.Client()

In [5]:
collection = chroma_client.create_collection(name='my_collection')

In [6]:
collection.add(
    documents=[
        'This is a document about pineapple',
        'This is a document about mango',
        'This is a document about strawberry',
    ],
    ids=['ids1','ids2','ids3']
)

In [7]:
results = collection.query(
    query_texts = ['This is a query document about vietnam'],
    n_results = 3
)

In [8]:
results

{'ids': [['ids1', 'ids2', 'ids3']],
 'embeddings': None,
 'documents': [['This is a document about pineapple',
   'This is a document about mango',
   'This is a document about strawberry']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[None, None, None]],
 'distances': [[1.2225853204727173, 1.2783520221710205, 1.3223111629486084]]}

# SciQ dataset 활용

In [9]:
from datasets import load_dataset

dataset = load_dataset('sciq', split='train')
dataset = dataset.filter(lambda x: x['support'] != '')
dataset

Dataset({
    features: ['question', 'distractor3', 'distractor1', 'distractor2', 'correct_answer', 'support'],
    num_rows: 10481
})

In [10]:
client = chromadb.Client()
collection = client.create_collection(name="sciq_support")
#client.delete_collection(name="sciq_support")

In [11]:
#!pip install transformers==4.40.02 sentence-transformers==2.7.0
# 임베딩 위해서 사용하ㅁ

In [12]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

c:\Users\Playdata\anaconda3\envs\vectordb_env\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
supports = dataset['support'][:100]
supports_embed = embedding_model.encode(supports).tolist()

In [14]:
print(len(supports_embed))
print(len(supports_embed[0]))

100
384


In [15]:
collection.add(
    ids=[str(i) for i in range(100)],
    embeddings=supports_embed,
    metadatas=[{'type':'support', 'text':text} for text in supports]
)

In [16]:
questions = dataset['question'][:3]
questions_embed = embedding_model.encode(questions).tolist()

In [17]:
results = collection.query(
    query_embeddings=questions_embed,
    n_results=1
)

In [18]:
results

{'ids': [['36'], ['1'], ['2']],
 'embeddings': None,
 'documents': [[None], [None], [None]],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'type': 'support',
    'text': 'Agents of Decomposition The fungus-like protist saprobes are specialized to absorb nutrients from nonliving organic matter, such as dead organisms or their wastes. For instance, many types of oomycetes grow on dead animals or algae. Saprobic protists have the essential function of returning inorganic nutrients to the soil and water. This process allows for new plant growth, which in turn generates sustenance for other organisms along the food chain. Indeed, without saprobe species, such as protists, fungi, and bacteria, life would cease to exist as all organic carbon became “tied up” in dead organisms.'}],
  [{'text': 'Without Coriolis Effect the global winds would blow north to south or south to north. But Coriolis makes them blow northeast to southwest or the re

In [19]:
for i, q in enumerate(questions):
    print("Question:", q)
    print("Support:", results['metadatas'][i][0]['text'])
    print()

Question: What type of organism is commonly used in preparation of foods such as cheese and yogurt?
Support: Agents of Decomposition The fungus-like protist saprobes are specialized to absorb nutrients from nonliving organic matter, such as dead organisms or their wastes. For instance, many types of oomycetes grow on dead animals or algae. Saprobic protists have the essential function of returning inorganic nutrients to the soil and water. This process allows for new plant growth, which in turn generates sustenance for other organisms along the food chain. Indeed, without saprobe species, such as protists, fungi, and bacteria, life would cease to exist as all organic carbon became “tied up” in dead organisms.

Question: What phenomenon makes global winds blow northeast to southwest or the reverse in the northern hemisphere and northwest to southeast or the reverse in the southern hemisphere?
Support: Without Coriolis Effect the global winds would blow north to south or south to north. 

## Chroma DB를 이용한 키워드 기반 검색

In [20]:
documents = [
    "인공지능은 인간의 작업을 자동화하는 기술이다.",
    "기계 학습은 데이터에서 패턴을 학습하여 예측하는 기술이다.",
    "벡터 데이터베이스는 유사도를 기반으로 데이터를 검색하는 DB이다.",
    "AI는 사람들이 하는 일을 대신 수행할 수 있도록 돕는 기술이다.",
    "머신 러닝은 컴퓨터가 데이터를 분석하여 스스로 배우는 알고리즘이다.",
    "벡터 DB는 데이터를 빠르게 검색하기 위해 유사도 기반으로 동작한다.",
    "인공지능은 알고리즘을 사용하여 문제를 해결하는 시스템이다.",
    "기계 학습을 통해 컴퓨터는 경험을 바탕으로 더 나은 결정을 내릴 수 있다.",
    "벡터 데이터베이스는 고속 검색과 유사도 검색에 강점을 가진 데이터베이스이다.",
    "AI 기술은 자동화된 시스템을 통해 사람들의 작업을 효율적으로 돕는다.",
    "기계 학습 모델은 주어진 데이터를 학습하여 미래의 결과를 예측한다.",
    "벡터 DB는 데이터의 유사도를 계산하여 관련성이 높은 정보를 빠르게 찾는다.",
    "인공지능은 복잡한 문제를 해결하기 위해 인간의 지능을 모방하는 시스템이다.",
    "기계 학습 알고리즘은 데이터를 통해 패턴을 발견하고, 이를 통해 예측을 개선한다.",
    "벡터 데이터베이스는 대량의 데이터를 효율적으로 처리하고 빠른 검색을 가능하게 한다.",
    "대형 언어 모델(LLM)은 자연어를 이해하고 생성할 수 있는 인공지능 모델이다.",
    "트랜스포머 모델은 대형 언어 모델에서 주로 사용되는 네트워크 아키텍처이다.",
    "자연어 처리(NLP)는 텍스트 데이터에서 의미를 추출하고 분석하는 기술이다.",
    "파인튜닝(fine-tuning)은 사전 학습된 모델을 특정 작업에 맞게 조정하는 과정이다.",
    "GPT는 대형 언어 모델 중 하나로, 텍스트 생성에 매우 뛰어난 성능을 보인다.",
    "언어 모델은 문법적 의미를 이해하고, 자연스러운 문장을 생성하는데 사용된다.",
    "BERT는 텍스트의 양방향 문맥을 학습하여 자연어 처리에서 뛰어난 성능을 발휘하는 모델이다.",
    "자연어 생성(NLG)은 기계가 인간처럼 문장을 생성하는 기술이다.",
    "LLM은 방대한 양의 텍스트 데이터를 학습하여 질문에 답하고 정보를 제공할 수 있다.",
    "어텐션 메커니즘은 트랜스포머 모델에서 핵심적인 역할을 하는 기술로, 입력 문장에 대한 중요한 부분에 집중한다.",
    "GPT 모델은 대화형 AI 시스템에서 자연스러운 대화 생성을 가능하게 한다."
]

In [21]:
client = chromadb.PersistentClient(path='./chroma_db')
collection = client.get_or_create_collection(name='ai_documents')

In [22]:
# 1. 임베딩 모델 로드
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# 2. 임베딩 처리 -> add
doc_embed = embedding_model.encode(documents).tolist()
collection.add(
    ids=[str(i) for i in range(len(documents))],
    embeddings=doc_embed,
    metadatas=[{'type':'description', 'text':document} for document in documents]
)

c:\Users\Playdata\anaconda3\envs\vectordb_env\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [23]:
# 3. 질의 -> 임베딩 처리 -> 검색 결과
questions = ["LLM", "AI", "기계 학습", "NLP"]
question_embedded = embedding_model.encode(questions).tolist()

results = collection.query(
    query_embeddings=question_embedded,
    n_results=3
)

#print(results)
for i, q in enumerate(questions):
    print("키워드:", q)
    for result in  results['metadatas'][i]:
        print("검색된 문서:", result['text'])
    print()

키워드: LLM
검색된 문서: 대형 언어 모델(LLM)은 자연어를 이해하고 생성할 수 있는 인공지능 모델이다.
검색된 문서: LLM은 방대한 양의 텍스트 데이터를 학습하여 질문에 답하고 정보를 제공할 수 있다.
검색된 문서: AI는 사람들이 하는 일을 대신 수행할 수 있도록 돕는 기술이다.

키워드: AI
검색된 문서: AI 기술은 자동화된 시스템을 통해 사람들의 작업을 효율적으로 돕는다.
검색된 문서: GPT 모델은 대화형 AI 시스템에서 자연스러운 대화 생성을 가능하게 한다.
검색된 문서: AI는 사람들이 하는 일을 대신 수행할 수 있도록 돕는 기술이다.

키워드: 기계 학습
검색된 문서: 기계 학습은 데이터에서 패턴을 학습하여 예측하는 기술이다.
검색된 문서: 기계 학습 모델은 주어진 데이터를 학습하여 미래의 결과를 예측한다.
검색된 문서: 기계 학습 알고리즘은 데이터를 통해 패턴을 발견하고, 이를 통해 예측을 개선한다.

키워드: NLP
검색된 문서: 자연어 처리(NLP)는 텍스트 데이터에서 의미를 추출하고 분석하는 기술이다.
검색된 문서: 자연어 생성(NLG)은 기계가 인간처럼 문장을 생성하는 기술이다.
검색된 문서: BERT는 텍스트의 양방향 문맥을 학습하여 자연어 처리에서 뛰어난 성능을 발휘하는 모델이다.



## 영화 추천 시스템

In [24]:
import pandas as pd
df = pd.read_csv('./data/tmdb_5000_movies.csv')
df.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [25]:
client = chromadb.PersistentClient(path='./chroma_db')
collection = client.get_or_create_collection(name='movies')

model = SentenceTransformer('all-MiniLM-L6-v2')

c:\Users\Playdata\anaconda3\envs\vectordb_env\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [26]:
movies = [
    {
        "id": str(idx),
        "title": row['title'],
        "overview": row['overview'] if pd.notna(row['overview']) else ""
    } for idx, row in df.iterrows() # 각 row 가져올 수 있게 함
]

movies

[{'id': '0',
  'title': 'Avatar',
  'overview': 'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'},
 {'id': '1',
  'title': "Pirates of the Caribbean: At World's End",
  'overview': 'Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems.'},
 {'id': '2',
  'title': 'Spectre',
  'overview': 'A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. While M battles political forces to keep the secret service alive, Bond peels back the layers of deceit to reveal the terrible truth behind SPECTRE.'},
 {'id': '3',
  'title': 'The Dark Knight Rises',
  'overview': "Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's crimes to protect the late attorney's reputation and is 

In [27]:
# add
# - ids, embeddings: 줄거리, metadatas: 줄거리 text, 제목
for movie in movies:
    overview_embed = model.encode(movie['overview']).tolist()
    collection.add(
        ids=[movie['id']],
        embeddings=[overview_embed],
        metadatas=[{'title':movie['title'], 'text':movie['overview']}]
    )

In [32]:
# query: 제목 입력 -> 이 제목의 영화의 줄거리와 우사한 영화 출력
input_title = "Avatar"
query_text = df.loc[df['title']==input_title, 'overview'].iloc[0]


query_embed = model.encode(query_text).tolist()

results = collection.query(
    query_embeddings=query_embed,
    n_results=5
)


print("키워드:", input_title)
print("-" * 30)

for result in results['metadatas'][0]:
    print('제목:', result['title'])
    print('줄거리:', result['text'])
    print()

키워드: Avatar
------------------------------
제목: Avatar
줄거리: In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.

제목: Alien: Resurrection
줄거리: Two hundred years after Lt. Ripley died, a group of scientists clone her, hoping to breed the ultimate weapon. But the new Ripley is full of surprises … as are the new aliens. Ripley must team with a band of smugglers to keep the creatures from reaching Earth.

제목: The Black Hole
줄거리: The explorer craft U.S.S. Palomino is returning to Earth after a fruitless 18-month search for extra-terrestrial life when the crew comes upon a supposedly lost ship, the magnificent U.S.S. Cygnus, hovering near a black hole. The ship is controlled by Dr. Hans Reinhardt and his monstrous robot companion, Maximillian. But the initial wonderment and awe the Palomino crew feel for the ship and its resistance to the power of the black hole turn to h

### (논문) pdf 내용 검색

In [ ]:
#!pip install PyPDF2

In [37]:
clinet = chromadb.PersistentClient(path='./chroma_db')
#client.delete_collection('papers')
collection = client.create_collection(name='papers')

In [42]:
papers = [
    {'id':'1', 'title': '딥러닝', 'path': './data/deep_learning.pdf'},
    {'id':'2', 'title': '자연어처리', 'path': './data/nlp_paper.pdf'}
]

In [39]:
import PyPDF2

def extract_text_from_pdf(path):
    with open(path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        text = " ".join([page.extract_text() for page in reader.pages if page.extract_text()])
        
    return text

In [43]:
for paper in papers:
    text = extract_text_from_pdf(paper['path'])
    
    embedding = model.encode(text).tolist()
    
    collection.add(
        ids=[paper['id']],
        embeddings=[embedding],
        metadatas=[{'title':paper['title']}],
        documents=[text]
    )

In [45]:
#collection.get()

collection.get(
    include = ['embeddings', 'documents', 'metadatas']
)

{'ids': ['1', '2'],
 'embeddings': array([[-9.85554159e-02, -7.41795003e-02,  1.97857730e-02,
          4.62038592e-02,  5.83831817e-02,  7.11069489e-03,
         -8.13849419e-02, -2.21503507e-02,  5.01135848e-02,
          1.65855251e-02, -2.28593424e-02,  5.13185821e-02,
         -5.76936528e-02, -4.42963978e-03, -7.13543594e-02,
         -2.76288465e-02,  1.62793510e-02,  2.03238651e-02,
         -7.00187758e-02,  5.10980114e-02,  5.85418381e-02,
          1.42441001e-02,  6.71361536e-02, -2.60970443e-02,
          5.34349866e-02, -2.24954691e-02, -4.90563288e-02,
         -1.66185498e-02, -1.54334260e-02, -6.88543245e-02,
          6.46846220e-02,  9.33331996e-02,  1.36579098e-02,
          2.46776380e-02,  1.24652453e-01,  6.97831511e-02,
          4.04871069e-02, -1.21630421e-02,  7.20558390e-02,
          6.68370724e-02, -2.70545725e-02, -4.22955193e-02,
         -3.09021790e-02,  3.65845696e-03,  2.53829919e-02,
         -1.93827704e-03,  2.02788357e-02, -4.23706360e-02,
      

In [47]:
query_text = 'Naural Language'
query_embed = model.encode(query_text).tolist()

results = collection.query(query_embeddings=[query_embed], n_results=1)

#results['metadatas'][0][0]['title']
results

{'ids': [['2']],
 'embeddings': None,
 'documents': [[" \n \n \nChowdhury, G. (2003) Natural language processing. Annual Review of \nInformation Science and Technology, 37. pp. 51-89. ISSN 0066-4200 \n \n \n \nhttp://eprints.cdlr.strath.ac.uk/2611/\n \n \n \nThis is an author-produced versio n of a paper published in The \nAnnual Review of Information Science and Technology  ISSN 0066-4200 . \nThis version has been peer-reviewed, but does not include the \nfinal publisher proof corrections, published layout, or pagination. \n \nStrathprints is designed to allow users to access the research \noutput of the University of St rathclyde. Copyright © and Moral \nRights for the papers on this site are retained by the individual \nauthors and/or other copyright  owners. Users may download \nand/or print one copy of any articl e(s) in Strathprints to facilitate \ntheir private study or for non-co mmercial research. You may not \nengage in further distribution of th e material or use it for any 